In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium

from scipy import stats

### Acquire the Merged.csv of the health and Yelp data

In [4]:
df=pd.read_csv('merged.csv')
pd.set_option('display.max_columns', None)

In [5]:
df=pd.DataFrame(df)
df.head()

,Unnamed: 0,inspection_date,inspector,number_of_violations,violations,zip_code,house_numbers,categories,transactions,distance,review_count,longitude,latitude,name,sector,district,total_score,rating,price,establishment_x,establishment_y,address_x,address_y,trimmed_name_x,trimmed_name_y,clean_address_x,clean_address_y,address_distance,name_distance
0,0,2019-01-02,Cicily Martinez,3,"['22', '29', '45']",78212,2524,"['Italian', 'Desserts', 'Seafood']",[],4950.353662,99.0,-98.493780,29.453730,Capparelli's On Main,35,D,97,3.5,$$,capparellis on main,Capparelli's On Main,2524 main av n,2524 n main ave,capparellis main,capparellis main,main,n maine,3,0
1,1,2019-03-06,RYAN DAVIS,1,['31'],78230,14439,"['French', 'Asian Fusion', 'Tapas/Small Plates']",[],9312.955170,102.0,-98.544540,29.573330,Frederick's Bistro,9,A,98,4.0,$$$,fredrick's bistro,Frederick's Bistro,14439 military hy nw,14439 nw military hwy,fredrick's bistro,fredericks bistro,military hwy,nw military hwy,3,2
2,2,2019-07-10,Jacob Johanson,1,['19'],78212,825,['International Grocery'],[],3151.827872,20.0,-98.507240,29.466688,Tokyo Mart,18,B,97,4.5,$,tokyo mart,Tokyo Mart,825 hildebrand w,825 w hildebrand ave,tokyo mart,tokyo mart,hildebrand,w hildebrande,3,0
3,3,2019-11-04,Cicily Martinez,3,"['10', '15', '40']",78201,914,['Mexican'],['delivery'],3194.340357,42.0,-98.508619,29.466156,Fina's Kitchen,35,D,93,4.0,$,finas kitchen,Fina's Kitchen,914 hildebrand w,914 w hildebrand ave,finas kitchen,finas kitchen,hildebrand,w hildebrande,3,0
4,4,2019-03-14,RYAN DAVIS,0,[],78230,13151,['Barbeque'],['delivery'],7935.733616,41.0,-98.537950,29.562850,Bill Miller Bar-B-Q,9,A,100,2.5,$,bill miller bbq #29,Bill Miller Bar-B-Q,13151 nw military hy,13151 nw military hwy,bill miller bbq #29,bill miller barbq,military hwy,nw military hwy,3,6


In [6]:
# Dropping redundant and unneccesary columns

df= df.drop(df[['transactions', 'price', 'establishment_x', 'establishment_y', 'address_x', 'address_y', 'trimmed_name_x', 'trimmed_name_y', 'clean_address_x', 'clean_address_y', 'address_distance', 'name_distance']], axis=1)
df.describe()

,Unnamed: 0,number_of_violations,zip_code,house_numbers,distance,review_count,longitude,latitude,sector,total_score,rating
count,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000,2409.000000
mean,1204.000000,4.989622,78220.360731,4494.122042,7806.578807,161.892902,-98.514704,29.486358,16.587796,94.044832,3.574720
std,695.562722,4.708035,14.884181,4491.037788,3305.930381,264.744464,0.059607,0.055306,9.608700,5.482347,0.769284
min,0.000000,0.000000,78201.000000,1.000000,811.199667,1.000000,-98.708442,29.354318,1.000000,56.000000,1.000000
25%,602.000000,2.000000,78209.000000,849.000000,5840.653743,22.000000,-98.564331,29.430632,8.000000,91.000000,3.000000
50%,1204.000000,4.000000,78216.000000,2727.000000,7940.300340,69.000000,-98.499740,29.486788,17.000000,95.000000,3.500000
75%,1806.000000,7.000000,78230.000000,7400.000000,9571.434425,186.000000,-98.480582,29.526893,22.000000,98.000000,4.000000
max,2408.000000,50.000000,78259.000000,22211.000000,20222.717722,2661.000000,-98.397780,29.661520,35.000000,100.000000,5.000000


In [7]:
#Dropped all establishments with less than 22 reviews 
df=  df[df['review_count'] >= 22] 


#### 2409 to 1809 entries

In [8]:
#grouped health scores into 4 groups
health_scores= {'health_scores' : []}
for x in df['total_score']:
#     print(x)
    if x > 99:
        health_scores['health_scores'].append('1')
    elif x>=96:
        health_scores['health_scores'].append('2')
    elif x >=92:
        health_scores['health_scores'].append('3')
    else :
        health_scores['health_scores'].append('4')
#df
df['health_scores']= health_scores['health_scores']  

## Checking the distribution of health scores

In [9]:
df['health_scores'].value_counts()

2    523
4    508
3    458
1    331
Name: health_scores, dtype: int64

In [10]:
#encode for the inspectors
df['inspector'].value_counts()

Lauren Schumacher     180
Jeffrey Carrizales    136
RYAN DAVIS            129
Jacob Johanson        128
Sumner Bumgardner     117
Bamba Njie             89
Diana Garcia           88
Deborah Liu            82
Bernard Capdeboscq     79
Cicily Martinez        77
Valerie Suarez         75
Benito Zuniga          72
Daisy Zamora           68
Nathan Balfour         64
Raul Jimenez           38
John Payelle           38
William Hudson         37
Kassandra Aguilar      35
Debra Hernandez        31
Erica Arreola          27
Mario Cavazos          27
Kathleen Prenzler      26
MICHELLE MORENO        26
William Emminger       26
Amanda Cantu           22
Kacie Patrick          20
Adan Zamora            18
Sean Gilbert           16
Fermin Garza           15
Victoria Paredes        7
Monica Caballero        7
Juanicia Page           6
Joe Mokry               5
Daniel Guzman           3
Maurico Ripley          3
Erica Llanas            2
Rebecca Vera            1
Name: inspector, dtype: int64

In [11]:
#Dropping inspectors that have not copmpleted AT LEAST 2 inspections
vdf = pd.DataFrame(df['inspector'].value_counts())
vdf['check'] = vdf['inspector'] > 1
over_20 = list(vdf.loc[vdf['check'], :].reset_index()['index'])

In [12]:
df['check'] = df['inspector'].isin(over_20)
new_df = df.loc[df['check'], :]
new_df = new_df.drop(columns = ['check', 'Unnamed: 0'])


## total entries is now 1819

In [13]:
#Encoding the inspector column for modeling.
label_enc= LabelEncoder()
new_df['inspector_encode'] = label_enc.fit_transform(new_df.inspector)

In [14]:
#OneHot Encoding
one_enc = OneHotEncoder()
X = one_enc.fit_transform(new_df.inspector_encode.values.reshape(-1,1)).toarray()
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [15]:
#Joining the encoded data to the main Df
dfOneHot = pd.DataFrame(X, columns = ["health_inspector_"+str(int(i)) for i in range(X.shape[1])])

In [16]:
new_df = new_df.reset_index()
new_df = new_df.reset_index()
new_df['index']= new_df['level_0']
new_df = new_df.drop(columns = ['level_0'])
dfOneHot = dfOneHot.reset_index()
enc_df=new_df.join(dfOneHot, on = 'index', lsuffix='_left', rsuffix='_right')

### Decision Tree Model

In [17]:
#### Decision tree including number of review, total violations, yelp score, and inspector
#training / test split
X= enc_df.drop(columns=[ 'index_left', 'inspector_encode', 'index_right', 'inspection_date','longitude', 'latitude',  'inspector', 'number_of_violations','violations', 'zip_code', 'house_numbers', 'categories', 'distance', 'name', 'sector', 'district', 'health_scores', 'total_score'])
y= enc_df['health_scores']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42, stratify=enc_df.health_scores)
X_train.head()


,review_count,rating,health_inspector_0,health_inspector_1,health_inspector_2,health_inspector_3,health_inspector_4,health_inspector_5,health_inspector_6,health_inspector_7,health_inspector_8,health_inspector_9,health_inspector_10,health_inspector_11,health_inspector_12,health_inspector_13,health_inspector_14,health_inspector_15,health_inspector_16,health_inspector_17,health_inspector_18,health_inspector_19,health_inspector_20,health_inspector_21,health_inspector_22,health_inspector_23,health_inspector_24,health_inspector_25,health_inspector_26,health_inspector_27,health_inspector_28,health_inspector_29,health_inspector_30,health_inspector_31,health_inspector_32,health_inspector_33,health_inspector_34,health_inspector_35
1650,941.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
318,162.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1269,382.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1083,110.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1187,260.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#Train Model

#Create the Decision Tree Object


clf = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state= 42, min_samples_split= 2)

In [19]:
#Fit the model to the training data

clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [20]:
#Estimate health group

y_pred = clf.predict(X_train)
y_pred

array(['4', '4', '2', ..., '2', '4', '2'], dtype=object)

In [21]:
#Estimate the probability of a health group
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.        , 0.        , 0.2962963 , 0.7037037 ],
       [0.        , 0.23728814, 0.23728814, 0.52542373],
       [0.4       , 0.6       , 0.        , 0.        ],
       ...,
       [0.12195122, 0.58536585, 0.14634146, 0.14634146],
       [0.10504202, 0.19747899, 0.32773109, 0.3697479 ],
       [0.17567568, 0.32162162, 0.25      , 0.2527027 ]])

In [22]:
#Create a confusion matrix
confusion_matrix(y_train, y_pred)

array([[ 54, 153,   0,  25],
       [ 15, 290,   0,  61],
       [  4, 204,   4, 108],
       [  3, 193,   0, 159]])

In [23]:
#Accuracy: number of correct predictions over the number of total instances that have been evaluated.
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.40


In [24]:
#Create a classification report
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           1       0.71      0.23      0.35       232
           2       0.35      0.79      0.48       366
           3       1.00      0.01      0.02       320
           4       0.45      0.45      0.45       355

    accuracy                           0.40      1273
   macro avg       0.63      0.37      0.33      1273
weighted avg       0.61      0.40      0.33      1273



In [23]:
#Test Model

#Compute the accuracy of the model when run on the test data

print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on test set: 0.40


In [25]:
#Testing the accuracy with chi square

#Health scores and inspector

contingency_table = pd.crosstab(enc_df.inspector, enc_df.health_scores)
chi2_stat, p_val, dof, ex = stats.chi2_contingency(contingency_table)
print("===Chi2 Stat===")
print(chi2_stat)
print("\n")
print("===Degrees of Freedom===")
print(dof)
print("\n")
print("===P-Value===")
print(p_val)
print("\n")


===Chi2 Stat===
626.7361204368792


===Degrees of Freedom===
105


===P-Value===
3.0398036978396203e-75




### Decision Tree Model- Binning based on probability

In [26]:
#created a table to show the probability of inspectors assigning a certain health score

contingency_table = pd.crosstab(df.inspector, df.health_scores, margins=True)
contingency_table = contingency_table[contingency_table.All > 20]
contingency_table['less_92'] = round(contingency_table['4']/contingency_table['All'],2)
contingency_table['92_to_95'] = round(contingency_table['3']/contingency_table['All'],2)
contingency_table['96_to_99'] = round(contingency_table['2']/contingency_table['All'],2)
contingency_table['100'] = round(contingency_table['1']/contingency_table['All'],2)
contingency_table.reset_index(inplace = True)
contingency_table

health_scores,inspector,1,2,3,4,All,less_92,92_to_95,96_to_99,100
0,Amanda Cantu,3,9,9,1,22,0.05,0.41,0.41,0.14
1,Bamba Njie,1,21,35,32,89,0.36,0.39,0.24,0.01
2,Benito Zuniga,11,18,21,22,72,0.31,0.29,0.25,0.15
3,Bernard Capdeboscq,9,21,21,28,79,0.35,0.27,0.27,0.11
4,Cicily Martinez,10,18,26,23,77,0.30,0.34,0.23,0.13
5,Daisy Zamora,14,36,11,7,68,0.10,0.16,0.53,0.21
6,Deborah Liu,8,34,30,10,82,0.12,0.37,0.41,0.10
7,Debra Hernandez,13,11,3,4,31,0.13,0.10,0.35,0.42
8,Diana Garcia,1,14,30,43,88,0.49,0.34,0.16,0.01
9,Erica Arreola,6,7,7,7,27,0.26,0.26,0.26,0.22


In [27]:
df = df.merge(contingency_table)

In [28]:
df = df[['name','review_count','rating','less_92','92_to_95','96_to_99','100', 'health_scores']]
df.head()

,name,review_count,rating,less_92,92_to_95,96_to_99,100,health_scores
0,Capparelli's On Main,99.0,3.5,0.3,0.34,0.23,0.13,2
1,Fina's Kitchen,42.0,4.0,0.3,0.34,0.23,0.13,3
2,Rolando's Super Tacos No 1,145.0,3.5,0.3,0.34,0.23,0.13,3
3,Rolando's Super Tacos No 1,145.0,3.5,0.3,0.34,0.23,0.13,3
4,Fina's Kitchen,42.0,4.0,0.3,0.34,0.23,0.13,3


## Splitting the Data

In [29]:
#training / test split
X= df.drop(columns=['name','health_scores'])
y= df['health_scores']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42, stratify=df.health_scores)
X_train.head()

,review_count,rating,less_92,92_to_95,96_to_99,100
549,361.0,4.5,0.30,0.20,0.38,0.12
1063,27.0,3.5,0.26,0.19,0.21,0.34
456,445.0,3.5,0.06,0.23,0.52,0.19
1520,93.0,3.0,0.11,0.27,0.32,0.30
1319,362.0,3.0,0.57,0.26,0.14,0.02


In [29]:
#Train Model

#Create the Decision Tree Object

clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state= 42, min_samples_split= 2)

In [30]:
#Fit the model to the training data

clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [31]:
#Estimate health group

y_pred = clf.predict(X_train)
y_pred

array(['4', '4', '2', ..., '2', '2', '4'], dtype=object)

In [32]:
#Estimate the probability of a health group
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.07065217, 0.22282609, 0.31521739, 0.39130435],
       [0.23809524, 0.17460317, 0.23809524, 0.34920635],
       [0.20434783, 0.40869565, 0.27826087, 0.10869565],
       ...,
       [0.19327731, 0.38655462, 0.15966387, 0.2605042 ],
       [0.10344828, 0.68965517, 0.20689655, 0.        ],
       [0.07065217, 0.22282609, 0.31521739, 0.39130435]])

In [33]:
confusion_matrix(y_train, y_pred)

array([[103,  73,   9,  35],
       [ 56, 164,   6, 111],
       [ 27,  90,  13, 176],
       [ 14,  58,   0, 266]])

In [34]:
#Accuracy: number of correct predictions over the number of total instances that have been evaluated.
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.45


In [35]:
#Test Model

#Compute the accuracy of the model when run on the test data

print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on test set: 0.44


In [36]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           1       0.52      0.47      0.49       220
           2       0.43      0.49      0.45       337
           3       0.46      0.04      0.08       306
           4       0.45      0.79      0.57       338

    accuracy                           0.45      1201
   macro avg       0.46      0.45      0.40      1201
weighted avg       0.46      0.45      0.40      1201



### Creating a Random Forrest Model and setting the estimator to one million, because why not

In [38]:
#Train Model

#Create the Random Forest Object
#
rf = RandomForestClassifier(max_depth=4, criterion='entropy', n_estimators=100000, random_state=42)
#Fit the model to the training data
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10000,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [39]:
#Print the Feature importance
print(rf.feature_importances_)

[0.13652375 0.08570995 0.28438243 0.10908794 0.14448945 0.23980648]


In [40]:
#Estimate the probability of the health group, using the training data
rf.score(X_train, y_train)

0.4646128226477935

In [41]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           1       0.52      0.50      0.51       220
           2       0.43      0.48      0.45       337
           3       0.42      0.11      0.17       306
           4       0.47      0.74      0.57       338

    accuracy                           0.46      1201
   macro avg       0.46      0.46      0.43      1201
weighted avg       0.46      0.46      0.43      1201



In [42]:
#Compute the accuracy of the model when run on the test data

print('Accuracy of random forest classifier on test set: {:.2f}'
     .format(rf.score(X_test, y_test)))

Accuracy of random forest classifier on test set: 0.44


# Creating a Confusion Matrix To Better Understand the Decision Tree Model

In [37]:
confusion_matrix(y_train,y_pred)

array([[103,  73,   9,  35],
       [ 56, 164,   6, 111],
       [ 27,  90,  13, 176],
       [ 14,  58,   0, 266]])

In [38]:
#Making the confusion matrix "pretty"
labels=(" 100 ", " 96-99 ", " 92-95 ", " <92 ")
pretty_cm=pd.DataFrame(confusion_matrix(y_train,y_pred),index=labels,columns=labels)
pretty_cm.T

,100,96-99,92-95,<92
100,103,56,27,14
96-99,73,164,90,58
92-95,9,6,13,0
<92,35,111,176,266


## Takeaway: There is correlation between inspectors and the scores they give.
### Further exploration is warranted.